<a href="https://colab.research.google.com/github/minit88/ML-DL/blob/main/Lab_09/_2_mnist_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [29]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device =='cuda':
  torch.cuda.manual_seed_all(777)

In [30]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [31]:
mnist_train = dsets.MNIST(root = 'MNIST_data/',train=True,transform = transforms.ToTensor(),download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',train=False,transform=transforms.ToTensor(),download=True)

In [32]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,batch_size=batch_size,shuffle=True,drop_last=True)

In [33]:
# nn layers
linear1 = torch.nn.Linear(784,256,bias = True)
linear2 = torch.nn.Linear(256,256,bias=True)
linear3 = torch.nn.Linear(256,10,bias=True)
relu = torch.nn.ReLU()

In [34]:
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)


Parameter containing:
tensor([[-0.6645,  0.6646,  0.7159,  ..., -0.3040, -0.8945, -0.7977],
        [-1.2045,  2.4545, -1.8073,  ..., -1.7139, -0.0451,  0.3864],
        [ 0.3414,  0.3114, -1.9218,  ..., -0.4525, -0.6849,  0.9663],
        ...,
        [ 1.3915, -0.4048,  0.2338,  ..., -0.8257, -0.8397,  1.9816],
        [ 1.0414,  0.2130, -0.0417,  ...,  1.7541, -0.6454, -0.0821],
        [-0.1162, -1.2692,  0.8201,  ..., -0.8303, -0.5022,  0.0583]],
       requires_grad=True)

In [35]:
model = torch.nn.Sequential(linear1,relu,linear2,relu,linear3).to(device)

In [36]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate) # SGD보다 효율적 Adam


In [37]:
total_batch = len(data_loader) # for 을 몇번 돌리는지
for epoch in range(training_epochs):
  avg_cost = 0
  for X,Y in data_loader:
    X=X.view(-1,28*28).to(device)
    Y=Y.to(device)

    hypothesis = model(X)
    cost = criterion(hypothesis,Y)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost +=cost/total_batch
  print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 128.846588135
Epoch: 0002 cost = 35.890373230
Epoch: 0003 cost = 23.033329010
Epoch: 0004 cost = 16.146936417
Epoch: 0005 cost = 11.686596870
Epoch: 0006 cost = 8.541748047
Epoch: 0007 cost = 6.444218636
Epoch: 0008 cost = 4.927516937
Epoch: 0009 cost = 3.656711340
Epoch: 0010 cost = 2.671477079
Epoch: 0011 cost = 2.088673115
Epoch: 0012 cost = 1.675091147
Epoch: 0013 cost = 1.261783004
Epoch: 0014 cost = 1.041191578
Epoch: 0015 cost = 0.888344646
Learning finished


In [51]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())
 


Accuracy: 0.944100022315979
Label:  8
Prediction:  8
